# 전체 파일 중에서 최종검수 완료한 라인넘버 제외하고 나머지만 뽑도록 하는 코드<br>
## 1. 입력, 출력 데이터<br>
#### input : 누적 작업 데이터셋<br>
#### output = 해야할 작업 데이터셋 = 전체 데이터셋 - 누적 작업 데이터셋<br><br><br><br>

## 2. 사용자 입력
#### input_repo : 입력 디렉토리 경로<br>
#### output_repo : 출력 디렉토리 경로<br>
#### database_repo : 누적 작업 데이터셋<br><br><br>

## 3. class 불러와서 start 하면..
start :<br>
- get_filelist : 입력 디렉토리의 파일리스트 추출<br>
- get_csvlist : 입력 디렉토리의 csv파일리스트 추출<br>
- print_howmuchleft : 작업 몇개 남았는지 프린트<br><br>

- generate_Todo_Task_list : 해야할 작업 번호 리스트로 추출<br>
- generate_Todo_Task : 해야할 작업 데이터프레임 형태로 생성<br>
- save_as_csv : 최종결과 csv 형식으로 저장<br><br>

In [1]:
import json
import csv
import pandas as pd
import os
from os import walk
import datetime
from tqdm.notebook import tnrange


class Todo_Task:
    
    # main
    def start(self, input_repo, output_repo, database_repo):
        processed = self.howmuchleft(input_repo, output_repo, database_repo)
        Todo_Task_lineNumber_list, database = self.generate_Todo_Task_list(database_repo, processed)
        Todo_Task = self.generate_Todo_Task(Todo_Task_lineNumber_list, database)
        self.save_as_csv(input_repo, output_repo, Todo_Task)
        
    
    
    
    
    # 작업 얼마나 남음?
    def howmuchleft(self, input_repo, output_repo, database_repo):
        file_list = self.get_filelist(input_repo)
        csv_list = self.get_csvlist(input_repo, file_list)
        processed = self.print_howmuchleft(database_repo, csv_list)
        
        return processed
        

    # 해당 경로의 파일리스트 추출
    def get_filelist(self, input_repo):
        file_list = []
        for (dirpath, dirnames, filenames) in walk(input_repo):
            file_list.extend(filenames)
            break
        
        return file_list
    
    
    # 해당 경로의 csv파일의 경로+파일명 리스트 추출
    def get_csvlist(self, input_repo, file_list):
        csv_list = []
        for i in range(len(file_list)):
            filename, file_extension = os.path.splitext(f'{input_repo}/{file_list[i]}')

            if file_extension == '.csv':
                csv_list.append(filename+file_extension)
                
        return csv_list
    
    
    # 작업 진행 정도 체크
    # 작업한 리스트 : processed_lineNumber_list
    # 누적 데이터베이스 리스트 : database_lineNumber_list
    def print_howmuchleft(self, database_repo, csv_list):
        processed = pd.concat([pd.read_csv(f) for f in csv_list])    # 라인넘버 기준 안합친파일 합치기
        processed_lineNumber_list = list(processed['lineNumber'])    # 작업한 lineNumber list

        database = pd.read_csv(f'{database_repo}/{database_filename}')    # 누적 데이터베이스 read
        database_lineNumber_list = list(database['lineNumber'])    # 누적 데이터베이스의 lineNumber List

        print('전체 38451개에서 태깅된 '+str(len(list(database['lineNumber'])))+'개 중 '+str(len(list(processed['lineNumber'])))+'개의 검수를 마쳤습니다.')
        
        return processed

        
    # 작업해야할 lineNumber_list 추출
    def generate_Todo_Task_list(self, database_repo, processed):
        database = pd.read_csv(f'{database_repo}/{database_filename}')    # 누적 데이터베이스 read
        processed_lineNumber_set = set(processed['lineNumber'])    # 검수작업 완료된 문장
        database_lineNumber_set = set(database['lineNumber'])    # 전체 데이터베이스

        Todo_Task_lineNumber_list = list(database_lineNumber_set.difference(processed_lineNumber_set))    # database - processed.. 즉 아직 작업하지 않은 문장들을 추출

        return Todo_Task_lineNumber_list, database
    
    # Todo_Task 생성
    def generate_Todo_Task(self, Todo_Task_lineNumber_list, database):
        COLUMNS = ['lineNumber', 'originalText', 'transcription']
        Todo_Task = pd.DataFrame(columns=COLUMNS)
        print('해야할 작업 리스트를 생성 중입니다...')
    
        for i in tnrange(len(Todo_Task_lineNumber_list)):
            for j in range(len(database)):
                if Todo_Task_lineNumber_list[i]==database['lineNumber'][j]:
                    Todo_Task = Todo_Task.append({'lineNumber':database['lineNumber'][j], 'originalText':database['originalText'][j], 'transcription':database['transcription'][j]},  ignore_index=True)
                else:
                    pass

        Todo_Task = Todo_Task.sort_values(by=['lineNumber'])    # 작업물 sort by lineNumber
        print('작업 리스트의 생성이 완료되었습니다.')
        
        return Todo_Task

        
    # csv형태로 저장
    def save_as_csv(self, input_repo, output_repo, Todo_Task):
        year = str(datetime.datetime.now().year)[2:]
        month = str(datetime.datetime.now().month)
        day = str(datetime.datetime.now().day)
        hour = str(datetime.datetime.now().hour)
        minute = str(datetime.datetime.now().minute)
        
        print(f'{output_repo}의 경로에 남은 작업_{year}{month}{day}_{hour}{minute}.csv의 이름으로 파일 생성이 완료되었습니다.')
        
        Todo_Task.to_csv(f'{output_repo}/남은 작업_{year}{month}{day}_{hour}{minute}.csv', index=False, encoding='utf-8-sig')

In [2]:
a=Todo_Task()

In [3]:
input_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/concat'              # 건드릴 필요 X
database_repo = '/Users/jaytsol/Desktop/Type23 (2)/오리지널/downloaded'    # 건드릴 필요 X
database_filename = '다운로드데이터_21117_1752.csv'                        # 매번 수정 필요

output_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/남은 작업'            # 건드릴 필요 X

In [4]:
a.start(input_repo, output_repo, database_repo)

전체 38451개에서 태깅된 39498개 중 51249개의 검수를 마쳤습니다.
해야할 작업 리스트를 생성 중입니다...


  0%|          | 0/3550 [00:00<?, ?it/s]

작업 리스트의 생성이 완료되었습니다.
/Users/jaytsol/Desktop/Type23 (2)/누적/남은 작업의 경로에 남은 작업_21118_1744.csv의 이름으로 파일 생성이 완료되었습니다.
